### Data

Завантажуємо спочатку тестові данні.

In [5]:
!curl https://raw.githubusercontent.com/vseloved/prj-nlp-2019/master/tasks/07-language-as-sequence/run-on-test.json --output run-on-test.json

In [1]:
import json

with open('run-on-test.json') as f:
    run_on_js = json.load(f)

Набираємо тренувальних данних з с корпусів NLTK.

In [2]:
from nltk.corpus import brown
from nltk.corpus import reuters
from nltk.corpus import gutenberg
from nltk.corpus import abc
from nltk.corpus import treebank
from nltk.corpus import conll2000

source = []

for fileid in reuters.fileids():
    source.append(reuters.sents(fileid))
    
for fileid in gutenberg.fileids():
    source.append(gutenberg.sents(fileid))    
    
for fileid in brown.fileids():
    source.append(brown.sents(fileid))    
    
for fileid in abc.fileids():
    source.append(abc.sents(fileid))        
    
for fileid in treebank.fileids():
    source.append(treebank.sents(fileid))   

Далі генеруємо фейкові run-on речення на основі зібраного корпусу і розставляємо теги.

In [4]:
import spacy

nlp = spacy.load('en_core_web_lg', disable = ['ner','parser'])

In [5]:
import random
from collections import namedtuple
from collections import defaultdict
import nltk

# визначає випадковим способом скільки реченнь склеювати, 
# пропорції приблизні як у наданних вище тестових данних
def rand_run_on_num():
    x = random.random()
    if x < 0.25:
        return 0
    elif 0.25 <= x < 0.97:
        return 1
    
    return 2

Token = namedtuple('Token', 'text point_after pos tag lemma dep token_')

random.seed(273757)

corpus = []

def tag_tokens(tokens):   
    
    doc = nlp(' '.join(tokens))
    
    return [Token(token.text, False, token.pos_, token.tag_, token.lemma_, token.dep_, token) for token in doc]


for sents in source:        
    
    run_on_num = rand_run_on_num()
    buff = []
    prev_offset = 0   
    
    for sent in sents[1:]:
        if len(sent) < 5:
            continue

        exclude = False
        for word in sent:
            if word == '?' or word == ';' or word == '...':
                exclude = True
                break
        if exclude:
            continue

        tokens = tag_tokens(sent)
        buff.extend(tokens)          

        if prev_offset > 0:                
            if buff[prev_offset - 1].text == '.':
                del buff[prev_offset - 1]
                buff[prev_offset - 2] = buff[prev_offset - 2]._replace(point_after = True)                

                if not buff[prev_offset - 1].text.isupper():                
                    lwr = random.random()                
                    if lwr < 0.93: #псуємо сase
                        buff[prev_offset - 1] = buff[prev_offset -1].\
                        _replace(text = buff[prev_offset - 1].text.lower())                                            

        prev_offset = len(buff)            

        if run_on_num == 0:            
            corpus.append(buff)
            run_on_num = rand_run_on_num()
            prev_offset = 0
            buff = []                               

        run_on_num -= 1

In [6]:
len(corpus)

18701

Перемішуємо корпус та розділяємо на тренувальні і тестові датасети.

In [7]:
for i in range(10):
    random.shuffle(corpus)

train_index = int(0.7 * len(corpus))

train_data = corpus[: train_index]
test_data = corpus[train_index: ]

Збераємо біграми та триграми на тестових данних, які також включають крапки на кінці речення. 

In [8]:
bigrams = defaultdict(float)
trigrams = defaultdict(float)

all_tokens = [token for sent in train_data for token in map(lambda x: x.text.lower(),sent)]

for g in nltk.ngrams(all_tokens, 2):        
    bigrams[g] += 1.0
                
for g in nltk.ngrams(all_tokens, 3):    
    trigrams[g] += 1.0

all_tokens = []

def update_freq(ngrams):
    n = sum(ngrams.values())
    for k, v in ngrams.items():
        ngrams[k] = v/n 

update_freq(bigrams)
update_freq(trigrams)        

In [9]:
sorted(bigrams.items(), key=lambda kv: kv[1], reverse = True)[:20]

[(('.', 'the'), 0.00598490423501983),
 (('of', 'the'), 0.0055116691522362525),
 (("'", 's'), 0.00521861072941156),
 (('in', 'the'), 0.004758400465420191),
 (('said', '.'), 0.003377769673446085),
 ((',', 'the'), 0.003186738997827026),
 (('said', 'the'), 0.0029913667159438976),
 ((',', '000'), 0.0029544630626993067),
 (('mln', 'dlrs'), 0.0028524353154936732),
 (('u', '.'), 0.0026852834743269966),
 (('the', 'company'), 0.002631013396026128),
 (('.', '"'), 0.002572401711461189),
 (('.', 's'), 0.002500765208104042),
 (('s', '.'), 0.002405249870294513),
 ((',', '"'), 0.002403079067162478),
 (('for', 'the'), 0.0021121914474698206),
 (('to', 'the'), 0.001901623543662449),
 ((',', 'and'), 0.0018191330246451279),
 (('said', 'it'), 0.0017995957964568152),
 (('1', '.'), 0.0017995957964568152)]

In [10]:
sorted(trigrams.items(), key=lambda kv: kv[1], reverse = True)[:20]

[(('u', '.', 's'), 0.0024942582132514795),
 (('.', 's', '.'), 0.0023640097425856057),
 (('the', 'company', 'said'), 0.0011678946203039998),
 (('the', 'u', '.'), 0.0011244784634153755),
 (('.', 'the', 'company'), 0.0008704939456169219),
 (('said', '.', 'the'), 0.0008031989024395538),
 ((',', '000', 'dlrs'), 0.0007098541651290112),
 ((',', '000', 'tonnes'), 0.0006664380082403866),
 (('he', 'said', '.'), 0.0006382175062627807),
 (('.', 'it', 'said'), 0.0006186802356628996),
 ((',', 'it', 'said'), 0.0005839473101519999),
 ((',', '"', 'he'), 0.0005665808473965502),
 ((',', 'he', 'said'), 0.0004927733806858885),
 (('mln', 'dlrs', 'in'), 0.0004819193414637323),
 (('mln', 'dlrs', ','), 0.0004797485336193011),
 (('.', '"', 'the'), 0.0004558696473305576),
 ((',', 'the', 'company'), 0.0004493572237972639),
 (('.', '5', 'pct'), 0.00042547833750852045),
 (('"', 'he', 'said'), 0.00041896591397522676),
 (('it', 'said', '.'), 0.0004059410669086394)]

### Baseline

Намагаємось побудувати простий безлайн на правилах, та перевіямо якість на тестових датасетах.

In [11]:
import math

def baseline(data):
    result_data = []
    for sent in data:
        result_sent = []
        last_point = 0
        for i, word in enumerate(sent):            
            if (i - last_point) > 3 and i < (len(sent) - 1):
                pbigram = (word[0].lower(), '.')                
                bigram = (word[0].lower(), sent[i+1][0].lower())                                    
                if (sent[i + 1][0][0:1].isupper()):
                    result_sent.append([word[0], True])
                elif math.log(bigrams[pbigram] + 0.000000000001) > math.log(bigrams[bigram] + 0.001):                        
                    result_sent.append([word[0], True])
                    last_point = i
                else:                       
                    result_sent.append([word[0], False])                                    
            else:                
                result_sent.append([word[0], False])
            
        result_data.append(result_sent)   
    
    return result_data

In [12]:
result = baseline(test_data)

In [13]:
# виділяє вектор лейблів з розміченних данних
def labels_vec(data):
    return [word[1] for sent in data for word in sent]

In [14]:
from sklearn.metrics import classification_report


print(classification_report(labels_vec(test_data), labels_vec(result)))

              precision    recall  f1-score   support

       False       0.99      0.91      0.95    196267
        True       0.02      0.18      0.03      1655

   micro avg       0.91      0.91      0.91    197922
   macro avg       0.51      0.55      0.49    197922
weighted avg       0.98      0.91      0.94    197922



In [15]:
print(classification_report(labels_vec(run_on_js), labels_vec(baseline(run_on_js))))

              precision    recall  f1-score   support

       False       0.98      0.96      0.97      4542
        True       0.26      0.45      0.33       155

   micro avg       0.94      0.94      0.94      4697
   macro avg       0.62      0.70      0.65      4697
weighted avg       0.96      0.94      0.95      4697



## Model

Далі будуємо і тренуємо модель базовану на логістичній регресії, порівнюємо 2 варіанта за n-грамами та без них.

In [22]:
import sys

def bigram_log_freq(word1, word2):
    return math.log((0.0 if (word1 is None) or (word2 is None)\
                    else bigrams[(word1.lower(), word2.lower())]) + 0.000001)

def trigram_log_freq(word1, word2, word3):
    return math.log((0.0 if (word1 is None) or (word2 is None) or (word3 is None)\
                    else trigrams[(word1, word2, word3)]) + 0.000001)

def get_prop(token, name, default):
    return default if token is None else getattr(token, name)

def extract_sent_features(sent, use_ngrams):
    features = []
    next1_token = sent[1] if len(sent) > 1 else None
    next2_token = sent[2] if len(sent) > 2 else None
    prev1_token = None
    prev2_token = None    
    for i, token in enumerate(sent):
        fdic = {}                
        fdic['next_case1'] = get_prop(next1_token, 'text', '')[:1].isupper()        
        fdic['case'] = token.text[:1].isupper()
        
        fdic['tag'] = token.tag
        fdic['tag_prev1'] = get_prop(prev1_token, 'tag', 'NONE_TAG')
        fdic['tag_next1'] = get_prop(next1_token, 'tag', 'NONE_TAG')
        fdic['tag_prev2'] = get_prop(prev2_token, 'tag', 'NONE_TAG')
        fdic['tag_next2'] = get_prop(next2_token, 'tag', 'NONE_TAG')
        
        fdic['index1'] = len(sent)/(i + 1)
        fdic['index2'] = len(sent)/(len(sent) - i +  1)
        
        fdic['word'] = token.text
        fdic['prev_word1'] = get_prop(prev1_token, 'text', '')
        fdic['next_word1'] = get_prop(next1_token, 'text', '')
        fdic['prev_word2'] = get_prop(prev2_token, 'text', '')
        fdic['next_word2'] = get_prop(next2_token, 'text', '')
                
        if use_ngrams:
            fdic['bigram'] = bigram_log_freq(token.text, get_prop(next1_token, 'text', None))
            fdic['pbigram'] = bigram_log_freq(token.text, '.')        
            fdic['pbigram2'] = bigram_log_freq('.', get_prop(next1_token, 'text', None))        
            fdic['ptrigram'] = trigram_log_freq(token.text, '.', get_prop(next1_token, 'text', None))         
            fdic['ptrigram2'] = trigram_log_freq('.', get_prop(next1_token, 'text', None), get_prop(next2_token, 'text', None)) 
            fdic['ptrigram3'] = trigram_log_freq(get_prop(prev2_token, 'text', None), get_prop(prev1_token, 'text', None), '.')        
        
        features.append(fdic)
        
        prev2_token = prev1_token
        prev1_token = token
        
        next1_token = next2_token
        next2_token = sent[i+3] if (i+3) < len(sent) else None
    
    return features

def extract_features(sents, use_ngrams):
    features = []
    for sent in sents:
        features.extend(extract_sent_features(sent, use_ngrams))
    return features     

In [29]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression 

class RunOnModel:
    def __init__(self, use_ngrams):
        self.use_ngrams = use_ngrams
        self.vectorizer = DictVectorizer()
        self.logreg = LogisticRegression(random_state=26, solver='liblinear', multi_class='ovr', max_iter=3000)
        
    def train(self, data):
        features = extract_features(data, self.use_ngrams)
        self.vectorizer.fit(features)
        feature_vecs = self.vectorizer.transform(features)
        self.logreg.fit(feature_vecs, labels_vec(data))
        
    def predict(self, data):
        vec = self.vectorizer.transform(extract_features(data, self.use_ngrams))
        return self.logreg.predict(vec)   

In [30]:
simple_mod = RunOnModel(use_ngrams = False)
simple_mod.train(train_data)

ngram_mod = RunOnModel(use_ngrams = True)
ngram_mod.train(train_data)

In [31]:
print(classification_report(labels_vec(test_data), simple_mod.predict(test_data)))

              precision    recall  f1-score   support

       False       1.00      1.00      1.00    196267
        True       0.84      0.45      0.59      1655

   micro avg       0.99      0.99      0.99    197922
   macro avg       0.92      0.73      0.79    197922
weighted avg       0.99      0.99      0.99    197922



In [32]:
print(classification_report(labels_vec(test_data), ngram_mod.predict(test_data)))

              precision    recall  f1-score   support

       False       1.00      1.00      1.00    196267
        True       0.69      0.65      0.67      1655

   micro avg       0.99      0.99      0.99    197922
   macro avg       0.85      0.82      0.83    197922
weighted avg       0.99      0.99      0.99    197922



Далі перевіряемо натреновану модель на наданих тестових данних, але перед цим тегаємо данні та виправяемо лейбли, оскліьки токенізація може відрізнятись.

In [33]:
def convert_to_tokens(data):
    tagged_sents = []
    
    for sent in data:
        tagged_sent = [] 
        tokens = tag_tokens(list(map(lambda w: w[0], sent)))
        compound = None
        merged = 0
        i = 0
        j = 0
        while i < len(tokens) or j < len(sent): 
            orig_token = sent[i] if i < len(sent) else sent[len(sent) - 1]
            token = tokens[j] if j < len(tokens) else tokens[len(tokens) - 1]           
            if orig_token[0] == token.text:                
                tagged_sent.append(token._replace(point_after = orig_token[1]))
                j += 1
                i += 1
            elif orig_token[0] in token.text:                
                if not compound:
                    compound = token
                else:
                    compound = compound._replace(point_after = orig_token[1])
                merged += len(orig_token[0])    
                if merged == len(compound.text):                     
                    j += 1
                    tagged_sent.append(compound)
                    compound = None                
                    merged = 0
                i += 1    
            elif token.text in orig_token[0]:                
                if not compound:
                    compound = orig_token[0]
                    
                merged += len(token.text)
                if merged == len(compound): 
                    i += 1
                    tagged_sent.append(token._replace(point_after = orig_token[1]))                    
                    compound = None
                    merged = 0
                else:
                    tagged_sent.append(token)
                j += 1
            else:                              
                assert False                                    
        
        tagged_sents.append(tagged_sent)
    
    return tagged_sents

run_on_js_tokenized = convert_to_tokens(run_on_js)

In [34]:
print(sum([w[1] for sent in run_on_js for w in sent]))
print(sum([t.point_after for sent in run_on_js_tokenized for t in sent]))

155
155


In [35]:
print(classification_report(labels_vec(run_on_js_tokenized), simple_mod.predict(run_on_js_tokenized)))

              precision    recall  f1-score   support

       False       0.97      1.00      0.99      4618
        True       0.78      0.14      0.23       155

   micro avg       0.97      0.97      0.97      4773
   macro avg       0.87      0.57      0.61      4773
weighted avg       0.97      0.97      0.96      4773



In [36]:
print(classification_report(labels_vec(run_on_js_tokenized), ngram_mod.predict(run_on_js_tokenized)))

              precision    recall  f1-score   support

       False       0.98      1.00      0.99      4618
        True       0.74      0.37      0.50       155

   micro avg       0.98      0.98      0.98      4773
   macro avg       0.86      0.68      0.74      4773
weighted avg       0.97      0.98      0.97      4773

